# Verifying the results of Unique SuperSet in Spark

In [1]:
import numpy as np
import pandas as pd
import copy

In [2]:
row_items = []
with open("testInp.txt") as f_1:
    for line in f_1:
        row_items.append(line.strip("\n").split("\t")[1].replace(","," "))

In [3]:
row_items_df = pd.DataFrame(row_items, columns=["Trans"])

In [4]:
with open("testInp_n.txt", "w+") as f_2:
    for x in row_items:
        f_2.write(x+"\n")

## Finding Unique SuperSet Trans

In [5]:
n = len(row_items)

unique_SS = []

for i in range(n):
    temp_1 = set(row_items[i].split(" "))
    # print("temp_1: ", temp_1)
    flag = True
    temp_uss = copy.deepcopy(unique_SS)
    for temp_2 in temp_uss:
        #temp_2 = set(temp_uss[j].split(" "))
        # print("temp_2: ", temp_2)
        if temp_1 == temp_2 or temp_1.issubset(temp_2):
            # print("In Condition 1")
            flag = False
            break
        elif temp_2.issubset(temp_1):
            # print("In Condition 2")
            unique_SS.remove(temp_2)          
            # break
        else:
            continue
    if flag:
        unique_SS.append(temp_1)
        
        #temp = ""
        #for y in sorted(list(map(int,list(temp_1)))):
        #    temp+=str(y)+" "
        #unique_SS.append(temp.strip(" "))
    # print(unique_SS)


In [6]:
len(unique_SS)

16

In [7]:
unique_SS = [" ".join(sorted(list(s))) for s in unique_SS]

In [8]:
unique_SS

['200 202 203 204 205 206 207 208 218 219 220',
 '203 204 205 206 207 208 209 213 216',
 '204 205 206 207 208 209 210',
 '205 206 207 208 209 210 211',
 '203 204 205 206 207 208 209 212 216 218',
 '201 202 203 204 205 206 207 211 218 219 220',
 '200 201 202 203 204 205 206 211',
 '200 202 204 206 208 211 212 213 214',
 '203 204 211 212 213 214 215 216 217 218 219 220',
 '206 207 209 212 213 214 215 216',
 '201 206 207 209 215 216 217 218 219 220',
 '201 202 203 209 215 216',
 '203 205 207 209 211 213 215 217',
 '207 209 211 213 216 218 219 220',
 '202 204 206 208 210',
 '210 212 214 216 218']

In [9]:
unique_SS_df = pd.DataFrame(unique_SS, columns=["CodeUnique_SS"])

In [10]:
unique_SS_df

,CodeUnique_SS
0,200 202 203 204 205 206 207 208 218 219 220
1,203 204 205 206 207 208 209 213 216
2,204 205 206 207 208 209 210
3,205 206 207 208 209 210 211
4,203 204 205 206 207 208 209 212 216 218
5,201 202 203 204 205 206 207 211 218 219 220
6,200 201 202 203 204 205 206 211
7,200 202 204 206 208 211 212 213 214
8,203 204 211 212 213 214 215 216 217 218 219 220
9,206 207 209 212 213 214 215 216


## Importing the results of uniqueSuperSets results from Spark

In [11]:
spark_res = pd.read_csv("testInp_USS.csv", header=None, names=["SparkUnique_SS"])

In [12]:
spark_res

,SparkUnique_SS
0,206 207 209 212 213 214 215 216
1,201 202 203 209 215 216
2,210 212 214 216 218
3,202 204 206 208 210
4,205 206 207 208 209 210 211
5,203 205 207 209 211 213 215 217
6,200 202 203 204 205 206 207 208 218 219 220
7,200 202 204 206 208 211 212 213 214
8,203 204 205 206 207 208 209 212 216 218
9,204 205 206 207 208 209 210


## Comparing both the results

In [13]:
merged_df = pd.merge(unique_SS_df, spark_res, how="inner", left_on="CodeUnique_SS", right_on="SparkUnique_SS")

In [14]:
merged_df

,CodeUnique_SS,SparkUnique_SS
0,200 202 203 204 205 206 207 208 218 219 220,200 202 203 204 205 206 207 208 218 219 220
1,203 204 205 206 207 208 209 213 216,203 204 205 206 207 208 209 213 216
2,204 205 206 207 208 209 210,204 205 206 207 208 209 210
3,205 206 207 208 209 210 211,205 206 207 208 209 210 211
4,203 204 205 206 207 208 209 212 216 218,203 204 205 206 207 208 209 212 216 218
5,201 202 203 204 205 206 207 211 218 219 220,201 202 203 204 205 206 207 211 218 219 220
6,200 201 202 203 204 205 206 211,200 201 202 203 204 205 206 211
7,200 202 204 206 208 211 212 213 214,200 202 204 206 208 211 212 213 214
8,203 204 211 212 213 214 215 216 217 218 219 220,203 204 211 212 213 214 215 216 217 218 219 220
9,206 207 209 212 213 214 215 216,206 207 209 212 213 214 215 216
